In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

<h3>URL CALL</h3>

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url)
results_page = BeautifulSoup(response.content, "html.parser")

<H3> Extract the table head and rows</H3>

In [4]:
table = results_page.find("table")
table_items = table.find_all("tr")
table_head = table_items[0].find_all("th")
table_rows = table_items[1:]

<h4>Create the dataframe with it being initialized as empty with requisite column names coming from table head.</h4>

In [5]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
toronto_neighborhoods = pd.DataFrame(columns = column_names)

<h4>Loop over the rows in table rows and append the applicable data in the dataframe. Ignore data where Borough is not assigned and assign Neighborhood to Borough if Borough is assigned while Neighborhood is not. If more than one neighborhood is assigned to the same postal code include all neighborhoods in same neighborhood column of dataframe for applicable postal code.</h4>

In [6]:
for row in table_rows:
    
    row = row.find_all("td")
    PostalCode = row[0].get_text()
    Borough = row[1].get_text()
    Neighborhood = row[2].get_text().strip("\n")
    
    if Neighborhood == "Not assigned" and Borough != "Not assigned":
        Neighborhood = Borough
    
    if Borough == "Not assigned":
        continue
    else:
        if PostalCode in toronto_neighborhoods["PostalCode"].tolist():
            index = toronto_neighborhoods["PostalCode"].tolist().index(PostalCode)
            toronto_neighborhoods.loc[[index], ["Neighborhood"]] = toronto_neighborhoods.loc[[index], ["Neighborhood"]] + ", " + Neighborhood 
        else:
            toronto_neighborhoods = toronto_neighborhoods.append({
                "PostalCode": PostalCode,
                "Borough": Borough,
                "Neighborhood": Neighborhood
            }, ignore_index = True)

In [7]:
toronto_neighborhoods

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."


In [8]:
toronto_neighborhoods.shape

(103, 3)

<h4>Read in a csv file containg latitude and longitude coordinates for the applicable toronto postal codes and store the data in a dataframe. Then join the two daataframes so all data is in one dataframe.</h4>

In [12]:
lat_long_df = pd.read_csv("./Geospatial_Coordinates.csv")
toronto_neigh_with_coord = toronto_neighborhoods.join(lat_long_df.set_index(["Postal Code"]), on = "PostalCode")
toronto_neigh_with_coord

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509


<h4>folium library</h4>

In [14]:
import folium

<h1>Create and show map of torronto</h1> 

In [22]:
map_Toronto = folium.Map(location=[43.715, -79.3832], zoom_start=10.5)

# add markers to map
for lat, lng, label in zip(toronto_neigh_with_coord['Latitude'], toronto_neigh_with_coord['Longitude'], toronto_neigh_with_coord['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7.5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False ).add_to(map_Toronto)  
    
map_Toronto

ValueError: ['parse_html'] are not valid options, expected ('bubbling_mouse_events', 'color', 'dash_array', 'dash_offset', 'fill', 'fill_color', 'fill_opacity', 'fill_rule', 'line_cap', 'line_join', 'opacity', 'stroke', 'weight')

In [23]:

CLIENT_ID = 'SQNBAVJBDG0POT5KFY2SVAOMWNUZB34ZVD4BR5XNFCNZXO3D' # your Foursquare ID
CLIENT_SECRET = '4KRJRO44XCYV0NLA5F5JROK2N5P3PQDODJEZYYFNCXQRJWRK' # your Foursquare Secret
VERSION = '20190601' # Foursquare API version

LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&time=any&day=any'.format(
    CLIENT_ID, CLIENT_SECRET, 43.657162, -79.378937, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=SQNBAVJBDG0POT5KFY2SVAOMWNUZB34ZVD4BR5XNFCNZXO3D&client_secret=4KRJRO44XCYV0NLA5F5JROK2N5P3PQDODJEZYYFNCXQRJWRK&ll=43.657162,-79.378937&v=20190601&radius=500&limit=100&time=any&day=any'

<h2>define function that creates data frame and return values</h2>

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&day=any&time=any'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h3>filter out neighbourhood in boroughs</h3>

In [25]:

toronto_boroughs = toronto_neigh_with_coord[toronto_neigh_with_coord['Borough'].str.contains('Toronto')]
toronto_boroughs

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
31,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259


<h4> Get near by Venues using the api call with lattitude and longitude </h4>

In [26]:
toronto_venues = getNearbyVenues(names=toronto_boroughs['Neighborhood'],
                                   latitudes=toronto_boroughs['Latitude'],
                                   longitudes=toronto_boroughs['Longitude']
                                  )

Harbourfront
Queen's Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

<h6>observe the venue dataframes and inspect first five observation </h6>

In [27]:
print(toronto_venues.shape)
toronto_venues.head()

(3185, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,Harbourfront,43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site


Inspect to see how many observation each neighbour has

In [28]:

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"Brockton, Exhibition Place, Parkdale Village",100,100,100,100,100,100
Business Reply Mail Processing Centre 969 Eastern,47,47,47,47,47,47
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",35,35,35,35,35,35
Central Bay Street,100,100,100,100,100,100
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,100,100,100,100,100,100


<h4> create data frame using one hot encoding venue..</h4>

In [29]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h4> create and inspect the frequency of each venue category </h4>

In [30]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.00,0.00,0.00,0.000000,0.000000,0.020000,0.00,0.00,0.00,...,0.010000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.010000,0.010000
1,Berczy Park,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,...,0.010000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.00,0.01,0.00,0.000000,0.000000,0.010000,0.00,0.00,0.00,...,0.020000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,...,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.00,0.00,0.00,0.066667,0.066667,0.000000,0.00,0.00,0.00,...,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000
5,"Cabbagetown, St. James Town",0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,...,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000
6,Central Bay Street,0.00,0.00,0.00,0.000000,0.000000,0.010000,0.00,0.00,0.00,...,0.020000,0.00,0.00,0.000000,0.00,0.010000,0.00,0.00,0.000000,0.010000
7,"Chinatown, Grange Park, Kensington Market",0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,...,0.070000,0.00,0.00,0.030000,0.00,0.010000,0.00,0.00,0.000000,0.020000
8,Christie,0.00,0.00,0.00,0.000000,0.000000,0.010000,0.00,0.00,0.00,...,0.020000,0.00,0.00,0.010000,0.00,0.010000,0.00,0.00,0.000000,0.000000
9,Church and Wellesley,0.00,0.00,0.01,0.000000,0.000000,0.010000,0.00,0.00,0.00,...,0.010000,0.00,0.01,0.010000,0.00,0.000000,0.01,0.00,0.000000,0.010000


<h6> see each venue and top five venues of neighbor </h6>

In [32]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0         Café  0.06
1  Coffee Shop  0.06
2        Hotel  0.05
3      Theater  0.04
4   Restaurant  0.03


----Berczy Park----
                 venue  freq
0          Coffee Shop  0.11
1                 Café  0.06
2                Hotel  0.05
3  Japanese Restaurant  0.04
4           Restaurant  0.04


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0                    Café  0.07
1             Coffee Shop  0.06
2                     Bar  0.05
3              Restaurant  0.04
4  Furniture / Home Store  0.04


----Business Reply Mail Processing Centre 969 Eastern----
              venue  freq
0       Coffee Shop  0.09
1              Park  0.09
2       Pizza Place  0.06
3           Brewery  0.06
4  Sushi Restaurant  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
             venue  freq
0      Coffee Shop  0.13
1             C

<h5>function to sort the most common venues</h5>

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<h4>Create and inspect new dataframe consisting of each nighborhood and its ten most common venues categories.</h4>

In [36]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Coffee Shop,Hotel,Theater,Restaurant,Steakhouse,Cosmetics Shop,Ramen Restaurant,Pub,Beer Bar
1,Berczy Park,Coffee Shop,Café,Hotel,Restaurant,Beer Bar,Japanese Restaurant,Park,Italian Restaurant,Breakfast Spot,Steakhouse
2,"Brockton, Exhibition Place, Parkdale Village",Café,Coffee Shop,Bar,Restaurant,Bakery,Furniture / Home Store,Gift Shop,Tibetan Restaurant,Lounge,Indian Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Park,Coffee Shop,Pizza Place,Brewery,Pet Store,Italian Restaurant,Sushi Restaurant,Burrito Place,Snack Place,Bistro
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Café,Coffee Shop,Harbor / Marina,Scenic Lookout,Dog Run,Park,Track,Dance Studio,Sculpture Garden,Garden


<h2> Import KMeans from sklearn </h2>

In [40]:
from sklearn.cluster import KMeans

<h2> create and fit model in data </h2

In [41]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering =toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 9, 8, 7, 8, 8, 9, 8, 8])

<h4> import matplotlib for mapping </h4>

In [47]:
import matplotlib.cm as cm
import matplotlib.colors as colors

<h5>Create and show map consisting of the neighborhoods in the merged dataframe with each bubble showing the location of the neighborhood and the color of bubble corresponding to the cluster it is assigned with neighborhoods assigned to the same cluster having the same color.

In [48]:
map_clusters = folium.Map(location=[43.675, -79.3832], zoom_start=12)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h4>From above map one can observe that in general neighborhoods that are a similar distance from the center of downtown toronto are are clustered together. Since the criteria for clustering was similarity in venues for the neighborhoods this implies that similar venues are within neighborhoods of similar distance from center of Toronto. This then also might imply that certain types of customers either live or frequent venues in those locations creating the demand for the same type of venues in those neighborhoods.
Below is each cluster one after another where you can look and inspect the neighborhoods in the cluster and compare the top 10 most common venue categories amongst the neighborhoods.

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,East Toronto,0,Greek Restaurant,Coffee Shop,Pub,Café,Ice Cream Shop,Italian Restaurant,Fast Food Restaurant,Pizza Place,Spa,Bakery
81,West Toronto,0,Café,Coffee Shop,Pizza Place,Bakery,Italian Restaurant,Diner,Falafel Restaurant,Sushi Restaurant,Restaurant,Pub


In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Downtown Toronto,1,Coffee Shop,Café,Seafood Restaurant,Hotel,Restaurant,Bakery,Breakfast Spot,Cosmetics Shop,Farmers Market,Beer Bar
20,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,Beer Bar,Japanese Restaurant,Park,Italian Restaurant,Breakfast Spot,Steakhouse
30,Downtown Toronto,1,Café,Coffee Shop,Hotel,Theater,Restaurant,Steakhouse,Cosmetics Shop,Ramen Restaurant,Pub,Beer Bar
36,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Aquarium,Brewery,Park,Scenic Lookout,Fried Chicken Joint
42,Downtown Toronto,1,Coffee Shop,Café,Hotel,Steakhouse,Restaurant,Italian Restaurant,Gym,Beer Bar,Seafood Restaurant,Japanese Restaurant
48,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,Steakhouse,Japanese Restaurant,Beer Bar,American Restaurant,Plaza,Concert Hall
92,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Japanese Restaurant,Beer Bar,Breakfast Spot,Gastropub,Hotel,Farmers Market,Seafood Restaurant
97,Downtown Toronto,1,Coffee Shop,Hotel,Café,Steakhouse,Theater,Italian Restaurant,Japanese Restaurant,Deli / Bodega,Concert Hall,Restaurant


In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,2,Coffee Shop,Diner,Café,Pub,Park,Restaurant,Italian Restaurant,Sushi Restaurant,Bakery,Breakfast Spot
67,Central Toronto,2,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Café,Sushi Restaurant,Dessert Shop,Gym,Pizza Place,Mexican Restaurant,Sporting Goods Shop
73,Central Toronto,2,Sporting Goods Shop,Skating Rink,Italian Restaurant,Coffee Shop,Café,Fast Food Restaurant,Mexican Restaurant,Diner,Park,Pizza Place
79,Central Toronto,2,Coffee Shop,Italian Restaurant,Sushi Restaurant,Pizza Place,Middle Eastern Restaurant,Gym,Dessert Shop,Pub,Indian Restaurant,Café
83,Central Toronto,2,Coffee Shop,Italian Restaurant,Grocery Store,Park,Thai Restaurant,Gym,Pub,Café,Sandwich Place,Pizza Place
86,Central Toronto,2,Coffee Shop,Park,Sushi Restaurant,Italian Restaurant,Grocery Store,Gym / Fitness Center,Thai Restaurant,Liquor Store,Sandwich Place,Café


In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Downtown Toronto,3,Park,Coffee Shop,Grocery Store,Playground,Athletics & Sports,Metro Station,Breakfast Spot,Candy Store,Bistro,Sandwich Place


In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,4,Café,Park,Bookstore,College Quad,Gym / Fitness Center,College Gym,Coffee Shop,Trail,Electronics Store,Doner Restaurant


In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,5,Bank,Café,Sushi Restaurant,Pharmacy,Coffee Shop,Italian Restaurant,Gastropub,Skating Rink,Lingerie Store,Bakery
68,Central Toronto,5,Café,Park,Coffee Shop,Gym / Fitness Center,Japanese Restaurant,Sushi Restaurant,Liquor Store,Trail,Burger Joint,Bank


In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
47,East Toronto,6,Indian Restaurant,Café,Coffee Shop,Beach,Burrito Place,Bakery,Sandwich Place,Fast Food Restaurant,Butcher,Burger Joint


In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 9, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,West Toronto,9,Café,Park,Coffee Shop,Sushi Restaurant,Italian Restaurant,Bar,Convenience Store,Brewery,Restaurant,Portuguese Restaurant
37,West Toronto,9,Café,Bar,Restaurant,Coffee Shop,Bakery,Pizza Place,Men's Store,Cocktail Bar,Vegetarian / Vegan Restaurant,Italian Restaurant
43,West Toronto,9,Café,Coffee Shop,Bar,Restaurant,Bakery,Furniture / Home Store,Gift Shop,Tibetan Restaurant,Lounge,Indian Restaurant
54,East Toronto,9,Coffee Shop,Bar,American Restaurant,Café,Bakery,Italian Restaurant,Brewery,Vietnamese Restaurant,French Restaurant,Diner
69,West Toronto,9,Café,Bar,Coffee Shop,Convenience Store,Thai Restaurant,Italian Restaurant,Sushi Restaurant,Park,Mexican Restaurant,Breakfast Spot
75,West Toronto,9,Coffee Shop,Bar,Sushi Restaurant,Pizza Place,Café,Sandwich Place,Restaurant,Breakfast Spot,Pub,Eastern European Restaurant
80,Downtown Toronto,9,Café,Bar,Vegetarian / Vegan Restaurant,Bakery,Bookstore,Restaurant,Mexican Restaurant,Coffee Shop,Pizza Place,Dessert Shop
84,Downtown Toronto,9,Café,Vegetarian / Vegan Restaurant,Bar,Coffee Shop,Art Gallery,Vietnamese Restaurant,Dessert Shop,Mexican Restaurant,Cocktail Bar,Tea Room
